<a href="https://colab.research.google.com/github/CloneTerial/ChatVoice/blob/main/ChatVoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://huggingface.co/CloneTerial/xtts-webui

Cloning into 'xtts-webui'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (30/30), 122.38 KiB | 2.45 MiB/s, done.
Filtering content: 100% (3/3), 1.74 GiB | 19.63 MiB/s, done.


In [ ]:
!sudo apt-get update # && sudo apt-get -y upgrade
!sudo apt-get -y install libegl1
!sudo apt-get -y install libopengl0
!sudo apt-get -y install libxcb-cursor
!pip install gradio==4.44.1
!pip install fastapi==0.103.1
!pip install pydantic==2.3.0
!pip install ctranslate2==4.4.0
!git clone https://huggingface.co/CloneTerial/xtts-webui
!cd xtts-webui
!pip install -r requirements.txt

In [ ]:
!pip install -q transformers==4.51.3 bitsandbytes==0.45.5 accelerate==1.5.2

In [ ]:

#custom fine tuning, skip jika hanya ingin tes
%cd /content/xtts-webui
!python xtts_demo.py --share

/content/xtts
* Running on local URL:  http://127.0.0.1:5003
* Running on public URL: https://f9eb6c13a2b26b42af.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Loading XTTS model! 
Model Loaded!
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2831, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/xtts/xtts_demo.py", line 1207, in <module>
    demo.laun

In [ ]:
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def load_xtts_model():
    xtts_checkpoint = "/content/xtts-webui/train_models/ready/model.pth"
    xtts_config = "/content/xtts-webui/train_models/ready/config.json"
    xtts_vocab = "/content/xtts-webui/train_models/ready/vocab.json"
    xtts_speaker = "/content/xtts-webui/train_models/ready/speakers_xtts.pth"

    clear_gpu_cache()
    config = XttsConfig()
    config.load_json(xtts_config)
    xtts_model = Xtts.init_from_config(config)
    xtts_model.load_checkpoint(
        config, checkpoint_path=xtts_checkpoint, vocab_path=xtts_vocab, speaker_file_path=xtts_speaker, use_deepspeed=False
    )
    if torch.cuda.is_available():
        xtts_model.cuda()
    return xtts_model

def run_tts(xtts_model, text, language, speaker_audio_file, temperature=0.8, length_penalty=1.0, repetition_penalty=5.0, top_k=50, top_p=0.8):
    gpt_cond_latent, speaker_embedding = xtts_model.get_conditioning_latents(
        audio_path=speaker_audio_file,
        gpt_cond_len=xtts_model.config.gpt_cond_len,
        max_ref_length=xtts_model.config.max_ref_len,
        sound_norm_refs=xtts_model.config.sound_norm_refs
    )

    out = xtts_model.inference(
        text=text,
        language=language,
        gpt_cond_latent=gpt_cond_latent,
        speaker_embedding=speaker_embedding,
        temperature=temperature,
        length_penalty=length_penalty,
        repetition_penalty=repetition_penalty,
        top_k=top_k,
        top_p=top_p,
        enable_text_splitting=True
    )

    output_path = "output.wav"
    out["wav"] = torch.tensor(out["wav"]).unsqueeze(0)
    torchaudio.save(output_path, out["wav"], 24000)

    return output_path

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True
)

base_model = "Qwen/Qwen2.5-14B-Instruct"  # Model dasar

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

# Load base model
ai_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",  # Auto Offload ke CPU jika GPU penuh
    # offload_folder="/content/offload"  # Direktori untuk menyimpan layer yang dialihkan ke RAM
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
import os
from IPython.display import display, Audio

prompt = "how are you"
messages = [
    {"role": "system", "content": "You are qwen ai"},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(ai_model.device)

generated_ids = ai_model.generate(
    **model_inputs,
    max_new_tokens=512,
)
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

xtts_model = load_xtts_model()
output_file = run_tts(xtts_model, response, "en", "/content/xtts-webui/train_models/ready/reference.wav")
print(f"Response: {response}")
display(Audio(output_file))

Response: I'm an artificial intelligence and don't have feelings, but I'm here to help you. How can I assist you today?
